# Investigating Spey fitting of a histogram from Contur

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import spey
spey.set_log_level(3) # debug

# load some data from a contur histogram
bg = np.array([3.612882e-03, 1.598201e-03, 3.186370e-05])
meas = np.array([3.2194e-03, 1.4437e-03, 3.0542e-05])
signal = np.array([1.26975467e-06, 0.00000000e+00, 0.00000000e+00])
measurement_cov = np.array([[1.25297487e-07, 3.41845046e-08, 1.09669171e-09],[3.41845046e-08, 1.19312634e-08, 3.20895254e-10],[1.09669171e-09, 3.20895254e-10, 1.88543549e-11]])
background_cov = np.array([[4.37507776e-08, 0.00000000e+00, 0.00000000e+00], [0.00000000e+00, 4.15880810e-09, 0.00000000e+00],[0.00000000e+00, 0.00000000e+00, 5.55546207e-13]])
signal_cov = np.array([[1.72952086e-12, 0.00000000e+00, 0.00000000e+00],[0.00000000e+00, 8.52595599e-15, 0.00000000e+00],[0.00000000e+00, 0.00000000e+00, 6.63785501e-17]])

## Recreate the calculation currently implemented in Contur

In [ ]:
from scipy.stats import norm

inv_cov_s_b = np.linalg.inv(signal_cov+measurement_cov+background_cov)
inv_cov_b_only = np.linalg.inv(measurement_cov+background_cov)

# contstruct a chi_square test statistic in the asymptotic limit
delta_mu_test = signal + bg - meas
delta_mu_null = bg - meas
ts_s_b = np.dot(delta_mu_test,np.dot(inv_cov_s_b,delta_mu_test))
ts_b = np.dot(delta_mu_null,np.dot(inv_cov_s_b,delta_mu_null))

# the test statistics are chi-sq distributed, so calculate the log(p-value)
log_pval_sb, log_pval_b = norm.logsf(np.sqrt((ts_s_b,ts_b)))

# convert to CLs value
CLs = 1 - np.exp(log_pval_sb - log_pval_b)
print(f'Current Contur CLs: {CLs}')

## Spey CLs Calculation

In [ ]:
# build the contur plugin
contur_model = spey.get_backend("contur.full_histogram_likelihood")(
    signal_yields=signal,
    background_yields=bg,
    data=meas,
    signal_covariance=signal_cov,
    background_covariance=background_cov,
    data_covariance=measurement_cov
)

In [ ]:
# build some default spey models for comparison
corr_bkd = spey.get_backend('default_pdf.correlated_background')(
    signal_yields=signal,
    background_yields=bg,
    data=meas,
    covariance_matrix=background_cov,
)

uncorr_bkd = spey.get_backend('default_pdf.uncorrelated_background')(signal_yields=signal,
                              background_yields=bg,
                              data=meas,
                              absolute_uncertainties=np.sqrt(np.diag(background_cov))
)

multinorm = spey.get_backend('default_pdf.multivariate_normal')(signal_yields=signal,
                              background_yields=bg,
                              data=meas,
                              covariance_matrix=signal_cov+background_cov+measurement_cov
)

### CLs computation for each model

All models give a CLs of 1.0 when the asymptotic calculator is used

The three models involving Poisson terms give a very low CLs value when the chi-squared calculator is used

Could this be because the Poisson terms expect yields larger than one?

In [ ]:
models = [contur_model,corr_bkd,uncorr_bkd,multinorm]
spey.set_log_level(2)
for model in models:
    print(model._backend.name,', asymptotic calculator, CLs: ' ,model.exclusion_confidence_level(calculator='asymptotic'))
    print(model._backend.name,', chi_square calculator, CLs: ' ,model.exclusion_confidence_level(calculator='chi_square'))

### Chi-squared calculator

**Conclusion:** For the models that have a Poisson term dependence, the optimiser isn't minimising the NLL very well, resulting in a weaker CLs value

From `spey.base.hypotest_base.HypothesisTestingBase`:

If `poi_test_denominator=None` computes

$$\chi^2 = -2\log\left(\frac{\mathcal{L}(\mu,\theta_\mu)}{\mathcal{L}(\hat\mu,\hat\theta)}\right)$$

else

$$\chi^2 = -2\log\left(\frac{\mathcal{L}(\mu,\theta_\mu)}{\mathcal{L}(\mu_{\rm denom},\theta_{\mu_{\rm denom}})}\right)$$

So what happens with no minimization of the NLL?

In [ ]:
# look at multivariate normal model first as that seems to give sensible results
# the poi that minimises the NLL is very close to 0, so these methods give the same results
spey.set_log_level(3)
print(f"without NLL minimization, CLs: {multinorm.exclusion_confidence_level(poi_test_denominator=0.0,calculator='chi_square')}")
print(f"with NLL minimization, CLs: {multinorm.exclusion_confidence_level(calculator='chi_square')}")


In [ ]:
# now try the uncorrelated background model
# this time, the poi that minimises the NLL is close to 1.0
spey.set_log_level(3)
print(f"without NLL minimization, CLs: {uncorr_bkd.exclusion_confidence_level(poi_test_denominator=0.0,calculator='chi_square')}")
print(f"with NLL minimization, CLs: {uncorr_bkd.exclusion_confidence_level(calculator='chi_square')}")

In [ ]:
# now try the contur one
# again the limit gets weaker when we try to do the NLL minimisation, so something is going wrong with the minimisation procedure
# also the value it finds is very close to the uncorrelated background case, suggesting that these Poisson based likelihoods are struggling to minimise the NLL
spey.set_log_level(3)
print(f"without NLL minimization, CLs: {contur_model.exclusion_confidence_level(poi_test_denominator=0.0,calculator='chi_square')}")
print(f"with NLL minimization, CLs: {contur_model.exclusion_confidence_level(calculator='chi_square')}")

### See how the likelihoods with Poisson terms perform when we set the initial poi guess to zero

*Conclusion:* They return a comparable exclusion to the multivariate normal model result

Q. the minimisation of NLL is not dependent on which calculator is used (right?)

From the log outputs, the poi is fit to a very small value ~1e-23 when `statistical_model.maximize_likelihood()` is called

Q. So why in `core.fit()` is the POI fixed to 1.0? 

In [ ]:
init_pars=[0.,1.,1.,1.]
corr_bkd.exclusion_confidence_level(calculator='chi_square',init_pars=init_pars,do_grad=True)


In [ ]:
contur_model.exclusion_confidence_level(calculator='chi_square',init_pars=list(np.append(0.0,np.ones(9))),do_grad=True)

### Asymptotic calculator

Q. Why is the limit 1.0 for all the models when this calculator is used?

Setting the initial POI to zero, we get something more sensible

Q. The initial parameter value is set to the user inputted value each time `core.fit()` is called, rather than using the `fit parameters` value returned by `statistical_model.maximize_likelihood()`, is this supposed to happen?

In [ ]:
multinorm.exclusion_confidence_level(calculator='asymptotic')

In [ ]:
# setting the initial POI to zero
multinorm.exclusion_confidence_level(calculator='asymptotic',init_pars=[0.0])